http://jmcauley.ucsd.edu/data/amazon/

http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/

https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt

In [4]:
import tensorflow as tf
import os
import json

import gzip
import shutil
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')
gitDir = "/content/gdrive/My Drive/nlp/"
os.chdir(gitDir + "data/")
print(os.listdir("."))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
['Kaggle Amazon Fine Food.ipynb', 'cache', '__pycache__', 'old', 'tmp.ipynb', '__init__.py', 'Organic Dataset.ipynb', 'amazon_multilingual.py', 'load_dataset.py', 'organic_dataset.py', 'Amazon Multilingual.ipynb', 'Amazon Reviews.ipynb']


In [0]:
def getReviewData(filename, baseurl="https://s3.amazonaws.com/amazon-reviews-pds/tsv/"):
  p = tf.keras.utils.get_file(filename,origin=baseurl + filename,
                                extract=False, cache_dir=".", cache_subdir="cache")
  outDir = p[0:-3]
  if os.path.exists(outDir) == False:
      with open(outDir, 'wb') as f_out, gzip.open(p, 'rb') as f_in:
          shutil.copyfileobj(f_in, f_out)
  else:
      pass
      print(outDir, "already exists. Skipping unzipping")

In [0]:
basePath = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/"
files = [
    ("amazon_reviews_us_Video_DVD_v1_00.tsv.gz",   "Video DVD"),
    ("amazon_reviews_us_Music_v1_00.tsv.gz",       "Music"),
    ("amazon_reviews_us_Books_v1_00.tsv.gz",       "Books"),
    ("amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz", "Mobile_Apps"),
    ("amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz", "Digital_Video_Download"),
    ("amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz", "Digital_Music_Purchase"),
    ("amazon_reviews_us_Toys_v1_00.tsv.gz", "Toys"),
    ("amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz", "Digital_Ebook_Purchase"),
    ("amazon_reviews_us_PC_v1_00.tsv.gz", "PC"),
    ("amazon_reviews_us_Camera_v1_00.tsv.gz", "Camera"),
    ("amazon_reviews_us_Wireless_v1_00.tsv.gz", "Wireless"),
    ("amazon_reviews_us_Electronics_v1_00.tsv.gz", "Electronics"),
    ("amazon_reviews_us_Video_v1_00.tsv.gz", "Video"),
    ("amazon_reviews_us_Sports_v1_00.tsv.gz", "Sports"),
    ("amazon_reviews_us_Video_Games_v1_00.tsv.gz", "Video Games"),
    ("amazon_reviews_us_Watches_v1_00.tsv.gz", "Watches"),
    ("amazon_reviews_us_Shoes_v1_00.tsv.gz", "Shoes"),
    ("amazon_reviews_us_Home_v1_00.tsv.gz", "Home"),
    ("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz", "Musical Instruments"),
    ("amazon_reviews_us_Baby_v1_00.tsv.gz", "Baby"),
    ("amazon_reviews_us_Home_Improvement_v1_00.tsv.g", "Home Improvement"),
    ("amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz", "Home Entertainment"),
    ("", "Office Products"),
    ("", ""),
    ("", ""),
    ("", ""),
    ("", ""),
    ("", ""),
    ("", ""),
    ("", ""),
    ("", ""),
    ("", ""),
    ("", ""),
    ("", ""),
    ("", ""),
    ("", ""),
    ("", ""),
    ("", ""),
    
    
]

for f in files:
  getReviewData(f[0], basePath)

2740338688/2740337188 [==============================] - 165s 0us/step


In [0]:
for f in files:
  print(f)
  num_lines = sum(1 for line in open("cache/"+f[0]))
  print("\t", num_lines)

In [0]:
def getCategory2IndexDict():
    d = {'Video DVD':                   0,
         'Music':                       1,
         'Books':                       2,
         'Mobile_Apps':                 3,
         'Digital_Video_Download':      4,
         'Digital_Music_Purchase':      5,
         'Toys':                        6,
         'Digital_Ebook_Purchase':      7,
         'PC':                          8,
         'Camera':                      9,
         'Wireless':                    10,
         'Electronics':                 11,
         'Video':                       12,
         'Sports':                      13,
         'Video Games':                 14,
         'Watches':                     15,
         'Shoes':                       16,
         'Home':                        17,
         'Musical Instruments':         18,
         'Baby':                        19,
         'Home Improvement':            20,
         'Home Entertainment':          21,
         'Office Products':             22,
         'Personal_Care_Appliances':    23,
         'Automotive':                  24,
         'Lawn and Garden':             25,
         'Luggage':                     26,
         'Kitchen':                     27,
         'Furniture':                   28,
         'Health & Personal Care':      29,
         'Software':                    30,
         'Grocery':                     31,
         'Pet Products':                32,
         'Beauty':                      33,

         #UK
         #'Apparel':                     34,

         #US
         #'Tools':                       35,
         #'Outdoors':                    36,
         #'Mobile_Electronics':          37,
         #'2012-12-22':                  38,
        }
    return d
  
  
def getCategory2CategoryDict():
    d = {"reviews_Books_5.json.gz":              "Books",
         "reviews_Electronics_5.json.gz":        "Electronics",
         "reviews_Movies_and_TV_5.json.gz":      "Video DVD",
         "reviews_CDs_and_Vinyl_5.json.gz":      "Music",
         "reviews_Clothing_Shoes_and_Jewelry_5.json.gz" : "Shoes",
         "reviews_Home_and_Kitchen_5.json.gz": "Home Improvement",
         "reviews_Kindle_Store_5.json.gz" : "Digital_Ebook_Purchase",
         "reviews_Sports_and_Outdoors_5.json.gz": "???",
         "reviews_Cell_Phones_and_Accessories_5.json.gz",
         "reviews_Health_and_Personal_Care_5.json.gz",
         "reviews_Toys_and_Games_5.json.gz",
         "reviews_Video_Games_5.json.gz",
         "reviews_Tools_and_Home_Improvement_5.json.gz",
         "reviews_Beauty_5.json.gz",
         "reviews_Apps_for_Android_5.json.gz",
         "reviews_Office_Products_5.json.gz",
         "reviews_Pet_Supplies_5.json.gz",
         "reviews_Automotive_5.json.gz",
         "reviews_Grocery_and_Gourmet_Food_5.json.gz",
         "reviews_Patio_Lawn_and_Garden_5.json.gz",
         "reviews_Baby_5.json.gz",
         "reviews_Digital_Music_5.json.gz",
         "reviews_Musical_Instruments_5.json.gz",
         "reviews_Amazon_Instant_Video_5.json.gz"]


        }
    return d

In [0]:
f = pd.read_json("cache/reviews_Musical_Instruments_5.json", orient="split")
f.head()

In [0]:
!head cache/reviews_Musical_Instruments_5.json

In [0]:
data = []
with open('cache/reviews_Musical_Instruments_5.json', 'r') as f:
    for line in f:
        j_content = json.loads(line)
        #print(j_content)
        #break
        data.append(j_content)
        #data.append(line)
        

In [0]:
f = pd.DataFrame.from_dict(data, orient='columns')
f.head()

In [0]:
ff = f[['summary','reviewText']]
ff.head()